In [26]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection 
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
# evaluate sklearn histogram gradient boosting algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
import random as rd
import timeit
import math
import sqlite3
from sqlalchemy import create_engine
from tsfresh import extract_features
from tsfresh import select_features
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from tsfresh import feature_extraction

In [2]:
from tsfresh import feature_extraction

In [3]:
os.chdir(r'C:\Users\formy\Downloads\amex-default-prediction')
csv_database = sqlite3.connect('csv_database.db') 
#pd.read_sql_query('SELECT customer_ID FROM data_use limit 1 ',csv_database)
#df = pd.read_sql_query('SELECT * FROM data_use where customer_ID = "00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7" ',csv_database)

In [4]:
# loading customer id's
import pickle
with open("train", "rb") as fp:   #UnPickling train
     train_customers = pickle.load(fp)
        
with open("test", "rb") as fp:   #UnPickling test
     test_customers = pickle.load(fp)

In [319]:
customer_slice = train_customers[10000:100000]
slice_query = 'SELECT * FROM raw_train_data WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
df = pd.read_sql_query(slice_query,csv_database)
df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
df_customers = df_d[['customer_ID','S_2']]
df_d = df_d.drop(['customer_ID','S_2'], axis=1)
imputer.fit(df_d)
df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
df_imp['index1'] = df_imp.index
df_customers['index1'] = df_customers.index
df_final = df_imp.merge(df_customers)
df_final = df_final.drop(['index','index1'], axis=1)

KeyboardInterrupt: 

In [ ]:
timeseries_cols = list(df_final.columns)
timeseries_cols = [i for i in timeseries_cols if i not in ['customer_ID','S_2']]

In [28]:
#column_id="id", column_sort="time", column_kind=None, column_value=None
col = [timeseries_cols[1],'customer_ID','S_2']
df_piece = df_final[col]
df_labels = pd.read_csv('train_labels.csv')
df_full = df_piece.merge(df_labels)
X = df_full.iloc[:,df_full.columns != 'target']
y = df_full[['customer_ID','target']]

NameError: name 'timeseries_cols' is not defined

In [ ]:
extracted_features = extract_features(X, column_id="customer_ID", column_sort="S_2",default_fc_parameters=kind_to_fc_parameters) 

In [84]:
extracted_features

,D_39__variance_larger_than_standard_deviation,D_39__has_duplicate_max,D_39__has_duplicate_min,D_39__has_duplicate,D_39__sum_values,D_39__abs_energy,D_39__mean_abs_change,D_39__mean_change,D_39__mean_second_derivative_central,D_39__median,...,D_39__permutation_entropy__dimension_6__tau_1,D_39__permutation_entropy__dimension_7__tau_1,D_39__query_similarity_count__query_None__threshold_0.0,"D_39__matrix_profile__feature_""min""__threshold_0.98","D_39__matrix_profile__feature_""max""__threshold_0.98","D_39__matrix_profile__feature_""mean""__threshold_0.98","D_39__matrix_profile__feature_""median""__threshold_0.98","D_39__matrix_profile__feature_""25""__threshold_0.98","D_39__matrix_profile__feature_""75""__threshold_0.98",D_39__mean_n_absolute_max__number_of_maxima_7
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.0,0.0,0.0,0.0,0.139148,0.008660,0.017597,0.000615,0.000121,0.002483,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018218
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.0,0.0,0.0,0.0,2.797663,1.078001,0.239676,-0.017055,0.001018,0.211731,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.362747
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.0,0.0,0.0,0.0,0.054349,0.000319,0.003603,0.000686,0.000246,0.004229,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006215
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.0,0.0,0.0,0.0,0.635204,0.124952,0.064904,-0.022283,0.011797,0.007425,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.089415
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.0,0.0,0.0,0.0,0.060366,0.000380,0.004048,-0.000083,-0.000031,0.004148,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
059da0f2ad23d7206b77787ebd4f8363d4664d001ec7516ad5ba5ee884c2cbbe,0.0,0.0,0.0,0.0,2.238757,1.271136,0.217571,0.000135,-0.018717,0.006612,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.317154
059dcec39ffc373f6057caa80aa7fd72f962a8dbc4d294e7f13f0d9484040175,0.0,0.0,0.0,0.0,1.589983,0.753608,0.250463,0.000042,-0.001981,0.006436,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.224570
059e38d9559ad3ff433427c8b91d3c3151d840b26504d6399b4274f9adbc1541,0.0,0.0,0.0,0.0,1.665613,1.010552,0.164659,0.075481,0.039931,0.031824,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.226677
059e5e843b4342ec4205e1703cb042bfadbd4df8ea0c221577cea6f7eb37bd32,0.0,0.0,0.0,0.0,1.606332,0.842104,0.227563,0.019133,0.010827,0.009660,...,2.079442,1.94591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.224522


In [85]:
y = y.groupby('customer_ID').mean()['target']

from tsfresh.utilities.dataframe_functions import impute
impute(extracted_features)
features_filtered = select_features(extracted_features, y.squeeze(),fdr_level=0.00000001,ml_task='classification')

In [87]:
features_filtered

,"D_39__linear_trend__attr_""stderr""",D_39__absolute_maximum,D_39__maximum,D_39__standard_deviation,D_39__variance,D_39__root_mean_square,"D_39__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0",D_39__quantile__q_0.9,"D_39__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0","D_39__fft_coefficient__attr_""abs""__coeff_2",...,"D_39__fft_coefficient__attr_""angle""__coeff_1","D_39__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.8",D_39__symmetry_looking__r_0.05,"D_39__fft_coefficient__attr_""angle""__coeff_2","D_39__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8",D_39__ratio_beyond_r_sigma__r_2.5,"D_39__fft_aggregated__aggtype_""kurtosis""",D_39__fourier_entropy__bins_100,D_39__approximate_entropy__m_2__r_0.3,D_39__last_location_of_minimum
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.001824,0.091505,0.091505,0.023485,0.000552,0.025809,0.000982,0.009741,0.001291,0.091603,...,-49.825087,0.0,0.0,-107.599346,0.0,0.076923,10.948788,1.747868,0.532397,0.769231
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.015407,0.567403,0.567403,0.191338,0.036610,0.287964,0.027799,0.505590,0.084953,0.760243,...,1.351457,0.0,1.0,169.107763,0.0,0.000000,3.387956,1.747868,0.292664,0.153846
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.000202,0.009704,0.009704,0.002650,0.000007,0.004950,0.000005,0.007363,0.000017,0.003787,...,35.292379,0.0,1.0,140.505005,0.0,0.000000,2.926819,1.747868,0.028513,0.384615
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.005300,0.268476,0.268476,0.084996,0.007224,0.098039,0.007209,0.184190,0.010925,0.255532,...,-34.114827,0.0,0.0,-60.078384,0.0,0.076923,6.350631,1.945910,0.279379,0.384615
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.000212,0.008680,0.008680,0.002769,0.000008,0.005407,0.000005,0.008042,0.000022,0.011936,...,-43.346679,0.0,0.0,-25.723022,0.0,0.000000,3.097993,1.747868,0.028513,0.461538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
059da0f2ad23d7206b77787ebd4f8363d4664d001ec7516ad5ba5ee884c2cbbe,0.018505,0.913673,0.913673,0.261003,0.068123,0.312697,0.073997,0.408348,0.121335,0.677176,...,117.008529,0.0,0.0,-121.687227,0.0,0.076923,4.988961,1.945910,0.079481,0.153846
059dcec39ffc373f6057caa80aa7fd72f962a8dbc4d294e7f13f0d9484040175,0.016590,0.596236,0.596236,0.207391,0.043011,0.240769,0.057126,0.492539,0.119858,0.935088,...,-175.082662,0.0,0.0,136.281749,0.0,0.000000,7.890339,1.945910,0.509247,0.076923
059e38d9559ad3ff433427c8b91d3c3151d840b26504d6399b4274f9adbc1541,0.016925,0.912104,0.912104,0.247627,0.061319,0.278810,0.062201,0.339265,0.083616,0.576374,...,48.012030,0.0,0.0,77.720992,0.0,0.076923,10.277806,1.945910,0.543996,0.153846
059e5e843b4342ec4205e1703cb042bfadbd4df8ea0c221577cea6f7eb37bd32,0.017861,0.743525,0.743525,0.222506,0.049509,0.254514,0.076777,0.430067,0.128196,0.773799,...,-69.997955,0.0,0.0,-67.911963,0.0,0.076923,9.980705,1.747868,0.543996,0.769231


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
sc = StandardScaler()
features_filtered_sc = sc.fit_transform(features_filtered[important_feautures])
pca = PCA(n_components=7)
features_filtered_pca = pca.fit_transform(features_filtered_sc)

In [88]:
def amex_metrics(model,X, y_true):
    y_pred = model.predict(X)
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

model = XGBClassifier()
rf = RandomForestRegressor(n_estimators=100)
scores = cross_val_score(model, features_filtered, y.squeeze(), cv=5,scoring = amex_metric)

scores

In [89]:
rf1 = RandomForestRegressor(n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(features_filtered, y.squeeze(), test_size=0.33, random_state=42)
rf1.fit(X_train, y_train)
sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
important_feautures = list(Importance_data.query('scores > 0.05')['feautures'])
important_feautures = list(Importance_data.query('scores > 0.1')['feautures'])
rf1.fit(X_train[important_feautures], y_train)
y_pred = rf1.predict(X_test[important_feautures])
y_true = y_test
amex_metric(y_true, y_pred)
#plt.xlabel("Random Forest Feature Importance")

In [98]:
#final_data = pd.merge(final_data,features_filtered[important_feautures],left_index=True, right_index=True) 

,B_2__root_mean_square,B_2__quantile__q_0.3,B_2__energy_ratio_by_chunks__num_segments_10__segment_focus_9,B_2__minimum,B_2__quantile__q_0.2,B_2__mean,B_2__quantile__q_0.1,D_39__quantile__q_0.7,"D_39__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","D_39__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2",...,D_39__energy_ratio_by_chunks__num_segments_10__segment_focus_1,D_39__mean_second_derivative_central,"D_39__fft_aggregated__aggtype_""variance""",D_39__minimum,D_39__median,D_39__time_reversal_asymmetry_statistic__lag_1,"D_39__cwt_coefficients__coeff_11__w_5__widths_(2, 5, 10, 20)","D_39__linear_trend__attr_""stderr""",D_39__maximum,D_39__absolute_maximum
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1.005091,1.003477,0.077315,1.000242,1.001516,1.005086,1.000668,0.005739,0.000894,0.001118,...,0.967629,0.000121,4.254506,0.001082,0.002483,2.325765e-06,0.008894,0.001824,0.091505,0.091505
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.992319,1.003475,0.078749,0.819772,1.002447,0.991083,1.001114,0.311601,-0.073233,-0.109891,...,0.340813,0.001018,5.370576,0.002224,0.211731,9.629095e-04,0.443115,0.015407,0.567403,0.567403
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.815684,0.812863,0.076352,0.810796,0.812213,0.815677,0.811217,0.005408,-0.000090,0.001377,...,0.184758,0.000246,5.591269,0.000802,0.004229,1.401861e-08,0.008204,0.000202,0.009704,0.009704
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.958428,0.965988,0.084780,0.812053,0.876689,0.955264,0.815977,0.018552,-0.037477,-0.016067,...,0.412419,0.011797,5.284181,0.000660,0.007425,-2.042496e-04,0.009864,0.005300,0.268476,0.268476
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.814549,0.812903,0.077149,0.810670,0.812035,0.814543,0.811139,0.006842,-0.000404,0.000401,...,0.073646,-0.000031,5.221589,0.000030,0.004148,-3.445963e-08,0.006385,0.000212,0.008680,0.008680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
059da0f2ad23d7206b77787ebd4f8363d4664d001ec7516ad5ba5ee884c2cbbe,0.909588,1.000160,0.094085,0.003914,0.889936,0.837688,0.183754,0.253024,-0.226765,-0.152431,...,0.000097,-0.018717,4.743939,0.001892,0.006612,-1.548159e-02,0.561923,0.018505,0.913673,0.913673
059dcec39ffc373f6057caa80aa7fd72f962a8dbc4d294e7f13f0d9484040175,0.410009,0.232066,0.118404,0.006665,0.181053,0.340460,0.086243,0.047432,-0.206141,-0.069628,...,0.000041,-0.001981,5.105096,0.000318,0.006436,-1.011474e-03,0.223943,0.016590,0.596236,0.596236
059e38d9559ad3ff433427c8b91d3c3151d840b26504d6399b4274f9adbc1541,0.025582,0.024229,0.007121,0.007783,0.022983,0.024951,0.021966,0.036321,0.248597,0.124740,...,0.002088,0.039931,4.220133,0.001514,0.031824,3.232482e-03,0.476542,0.016925,0.912104,0.912104
059e5e843b4342ec4205e1703cb042bfadbd4df8ea0c221577cea6f7eb37bd32,0.074040,0.022417,0.005970,0.020187,0.021176,0.049087,0.020645,0.034396,0.007340,-0.118641,...,0.656549,0.010827,4.255730,0.001490,0.009660,-4.882704e-05,0.157605,0.017861,0.743525,0.743525


In [102]:
# final_data = pd.DataFrame(index=final_data.index) # start from i = 94

In [ ]:
rf1 = lgb.LGBMClassifier(boosting_type = 'DART',min_child_samples = 500,n_jobs = 6,importance_type = 'gain',n_estimators = 100)
final_data_l = final_data.merge(df_labels,left_index=True,right_on=['customer_ID'])
final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
import re
final_data_l  = final_data_l.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', ' ', x))
final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
X = final_data_l.iloc[:,final_data_l.columns != 'target']
X = X.drop(['customer_ID'],axis =1)
#X= X[important_feautures]
y = final_data_l[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y.squeeze(), test_size=0.33, random_state=300)

rf1.fit(X_train, y_train)
y_pred = rf1.predict_proba(X_test)[:,1]
y_true = y_test
print('Column number:' + str(i)+ ' Metric for this column: ' +str(amex_metric(y_true, y_pred)))


sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
important_feautures = list(Importance_data.query('scores > 0.03')['feautures'])

#final_data.to_csv('final_data_tsfresh.csv')

In [316]:
### kind_to_fc_parameters = feature_extraction.settings.from_columns(final_data)

In [321]:
kind_to_fc_parameters

{'P_2': {'change_quantiles': [{'f_agg': 'mean',
    'isabs': False,
    'qh': 1.0,
    'ql': 0.0}],
  'mean_change': None,
  'quantile': [{'q': 0.2}, {'q': 0.3}, {'q': 0.1}],
  'minimum': None,
  'mean': None,
  'root_mean_square': None},
 'D_39': {'benford_correlation': None,
  'median': None,
  'cwt_coefficients': [{'coeff': 12, 'w': 5, 'widths': (2, 5, 10, 20)},
   {'coeff': 11, 'w': 5, 'widths': (2, 5, 10, 20)}],
  'minimum': None,
  'time_reversal_asymmetry_statistic': [{'lag': 1}],
  'linear_trend': [{'attr': 'stderr'}],
  'absolute_maximum': None,
  'maximum': None},
 'B_1': {'index_mass_quantile': [{'q': 0.9}],
  'cwt_coefficients': [{'coeff': 12, 'w': 2, 'widths': (2, 5, 10, 20)}],
  'median': None,
  'quantile': [{'q': 0.4}, {'q': 0.6}]},
 'B_2': {'quantile': [{'q': 0.3}, {'q': 0.2}, {'q': 0.1}],
  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 9}],
  'minimum': None,
  'mean': None},
 'R_1': {'root_mean_square': None, 'mean': None},
 'S_3': {'linear_trend':

In [294]:
extract_features(df, default_fc_parameters=settings)

Index(['P_2__change_quantiles__f_agg_ mean __isabs_False__qh_1 0__ql_0 0',
       'P_2__mean_change', 'P_2__quantile__q_0 2', 'P_2__minimum', 'P_2__mean',
       'P_2__quantile__q_0 3', 'P_2__root_mean_square', 'P_2__quantile__q_0 1',
       'D_39__benford_correlation', 'D_39__median',
       ...
       'D_68_5 0__fft_coefficient__attr_ angle __coeff_2',
       'D_68_5 0__fft_coefficient__attr_ angle __coeff_4',
       'D_68_5 0__range_count__max_1000000000000 0__min_0', 'D_68_5 0__length',
       'D_68_6 0__cwt_coefficients__coeff_0__w_2__widths_ 2 5 10 20 ',
       'D_68_6 0__linear_trend__attr_ intercept ', 'D_68_6 0__length',
       'D_68_6 0__cwt_coefficients__coeff_0__w_5__widths_ 2 5 10 20 ',
       'D_68_6 0__range_count__max_1000000000000 0__min_0',
       'D_68_6 0__cwt_coefficients__coeff_1__w_2__widths_ 2 5 10 20 '],
      dtype='object', length=12829)

MIN_SAMPLES = 200, 0.7399
MIN_SAMPLES = 20, 0.742
5, 0.73
50,  0.74
30, 0.741
25,

In [178]:
sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
    # important_feautures = list(Importance_data.query('scores > 0.05')['feautures'])
important_feautures = list(Importance_data.query('scores > 0.2')['feautures'])

In [134]:
from sklearn.model_selection import cross_val_predict
rf1 = RandomForestClassifier(n_estimators=100)
probas = cross_val_predict(rf1, X, y.squeeze(), cv=5, method='predict_proba')

In [ ]:
final_data # run until here

In [58]:
def accuracy(y_true, y_pred):
    return sum([y_true[i] == y_pred[i] for i in range(len(y_true))])/len(y_true)

In [ ]:
scores = cross_val_score(rf, features_filtered[important_feautures], y.squeeze(), cv=5,scoring = amex_metric)

In [ ]:
scores

In [ ]:
def feautures(nrows = 10000):
    customer_slice = train_customers[0:nrows]
    slice_query = 'SELECT * FROM raw_train_data WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
    df = pd.read_sql_query(slice_query,csv_database)
    df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
    df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
    df_customers = df_d[['customer_ID','S_2']]
    df_d = df_d.drop(['customer_ID','S_2'], axis=1)
    imputer.fit(df_d)
    df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
    df_imp['index1'] = df_imp.index
    df_customers['index1'] = df_customers.index
    df_final = df_imp.merge(df_customers)
    df_final = df_final.drop(['index','index1'], axis=1)

In [ ]:
feautures = {}

for col in df_final.columns:
    colums = [col,'customer_ID','S_2']
    df_piece = df_final[colums]
    df_labels = pd.read_csv('train_labels.csv')
    df_full = df_piece.merge(df_labels)
    X = df_full.iloc[:,df_full.columns != 'target']
    y = df_full[['customer_ID','target']]
    extracted_features = extract_features(X, column_id="customer_ID", column_sort="S_2")
    impute(extracted_features)
    features_filtered = select_features(extracted_features, y.squeeze(),fdr_level=0.00000001,ml_task='classification')
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(extracted_features, y.squeeze())
    sorted_idx = rf.feature_importances_.argsort()
    # plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
    max_importance = max(rf.feature_importances_[sorted_idx])
    Importance_data = pd.DataFrame()
    Importance_data['feautures'] = extracted_features.columns[sorted_idx]
    Importance_data['scores'] = [i/max_importance for i in rf.feature_importances_[sorted_idx]]
    important_feautures = list(Importance_data.query('scores > 0.05')['feautures'])
    sc = StandardScaler()
    features_filtered_sc = sc.fit_transform(features_filtered[important_feautures])
    pca = PCA(n_components=7)
    features_filtered_pca = pca.fit_transform(features_filtered_sc)

In [39]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [322]:
customer_slice = train_customers[0:10000]
slice_query = 'SELECT * FROM raw_train_data WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
df = pd.read_sql_query(slice_query,csv_database)
df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
df_customers = df_d[['customer_ID','S_2']]
df_d = df_d.drop(['customer_ID','S_2'], axis=1)
imputer.fit(df_d)
df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
df_imp['index1'] = df_imp.index
df_customers['index1'] = df_customers.index
df_final = df_imp.merge(df_customers)
df_final = df_final.drop(['index','index1'], axis=1)

In [327]:
extracted_features = extract_features(df_final, column_id="customer_ID", column_sort="S_2",kind_to_fc_parameters=kind_to_fc_parameters) 

Feature Extraction: 100%|███████████████████████████████████████████████████████████| 30/30 [2:11:32<00:00, 263.07s/it]


In [339]:
rf1 = lgb.LGBMClassifier(boosting_type = 'DART',min_child_samples = 500,n_jobs = 6,importance_type = 'gain',n_estimators = 100)
final_data_l = extracted_features.merge(df_labels,left_index=True,right_on=['customer_ID'])
final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
import re
final_data_l  = final_data_l.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '1', x))
#final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
X = final_data_l.iloc[:,final_data_l.columns != 'target']
X = X.drop(['customer_ID'],axis =1)
#X= X[important_feautures]
y = final_data_l[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y.squeeze(), test_size=0.33, random_state=300)

In [369]:
rf1.fit(X_train, y_train)
y_pred = rf1.predict_proba(X_test)[:,1]
y_true = y_test
print('Column number:' + str(i)+ ' Metric for this column: ' +str(amex_metric(y_true, y_pred)))
sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
ColumnMap = {}
for col in extracted_features.columns:
    ColumnMap[re.sub('[^A-Za-z0-9_]+', '1', col)] = col
Importance_data['feauture_v1'] = Importance_data[['feautures']].apply(lambda x:ColumnMap[x[0]] ,axis = 1)
important_feautures = list(Importance_data.query('scores > 0.001')['feauture_v1'])

Column number:197 Metric for this column: 0.71710767903393


In [368]:
kind_to_fc_parameters_score_1perc = feature_extraction.settings.from_columns(extracted_features[important_feautures])

In [13]:
extracted_features  = pd.read_sql_query('select * from train_tsfresh_imp',csv_database)
extracted_features = extracted_features.set_index(['index'])
kind_to_fc_parameters_score_point1perc = feature_extraction.settings.from_columns(extracted_features)

In [12]:
extracted_features

,index,D_56__permutation_entropy__dimension_3__tau_1,"R_3__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0",D_79__quantile__q_0.7,S_12__autocorrelation__lag_8,D_129__max_langevin_fixed_point__m_3__r_30,D_129__quantile__q_0.9,D_54__energy_ratio_by_chunks__num_segments_10__segment_focus_1,D_83__autocorrelation__lag_9,B_5__spkt_welch_density__coeff_5,...,D_39__absolute_maximum,"B_1__cwt_coefficients__coeff_12__w_2__widths_(2, 5, 10, 20)",D_48__mean,D_44__quantile__q_0.9,B_3__mean_change,R_1__mean,P_2__quantile__q_0.2,P_2__minimum,P_2__quantile__q_0.1,P_2__mean
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.720193,0.000000,0.004903,0.211674,1.006969,1.008457,0.154486,-0.801398,0.000262,...,0.091505,0.006067,0.274552,0.007706,0.000205,0.004509,0.922758,0.868580,0.910803,0.933824
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.641735,0.001713,0.007028,0.030642,0.004905,0.009025,0.153201,-0.536296,0.006700,...,0.567403,0.029178,0.048203,0.008515,-0.000064,0.006246,0.879521,0.861109,0.874581,0.899820
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,1.168518,0.000000,0.005263,-0.343246,0.010861,0.008118,0.153658,-0.177329,0.000021,...,0.009704,0.004172,0.092284,0.008313,0.000198,0.006621,0.868611,0.797670,0.855280,0.878454
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,1.294545,0.003590,0.007423,-0.906943,1.009527,1.008506,0.153553,0.082370,0.003635,...,0.268476,0.002133,0.172003,0.008605,-0.000082,0.005665,0.581358,0.567442,0.570931,0.598969
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,1.767761,0.000000,0.006361,-0.541641,1.006573,1.008552,0.154735,0.483887,0.000035,...,0.008680,0.004288,0.253697,0.007276,-0.000033,0.004180,0.863347,0.805045,0.860932,0.891679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,379c691f591070e27fbc48af22b7d1c5a3aaed252513f8...,1.277034,0.194552,0.007648,0.126560,1.008777,1.007205,0.111385,NaN,NaN,...,0.242714,NaN,0.354639,0.133499,0.000966,0.005954,0.436212,0.392494,0.420337,0.559006
99996,379cd985e084f4a891f2dd8ef293bed05a19171ed0cc13...,1.641735,0.000000,0.007743,0.167650,1.013576,1.008971,0.153219,0.052721,0.000014,...,0.506571,-0.010621,0.097014,0.008315,-0.000339,0.002651,0.998370,0.991149,0.993254,1.001864
99997,379ceebf8834f6272f58b71aa7124c1114e768940a7d04...,1.546599,0.001044,0.008398,-0.777252,1.009556,1.008985,0.153613,-0.754606,0.001824,...,0.474155,-0.039638,0.620484,0.134185,0.001455,0.043770,0.729619,0.665016,0.699169,0.752254
99998,379d1091601a628432bf15d688db67273a2332a787dc61...,0.600166,0.000000,0.008458,-0.248493,1.006003,1.007260,0.154771,-0.047571,0.002935,...,0.743441,-0.013585,0.164429,0.009093,0.000051,0.005393,0.883174,0.872863,0.878719,0.911351


In [20]:
customer_slice = train_customers[400000:]
slice_query = 'SELECT * FROM raw_train_data WHERE customer_ID in ('+"'" + "','".join(customer_slice)+"')"
df = pd.read_sql_query(slice_query,csv_database)
df_dum = pd.get_dummies(df,columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #, weights='uniform', metric='nan_euclidean' 
df_d = df_dum.dropna(axis='columns', thresh=0.4*len(df_dum.index))
df_customers = df_d[['customer_ID','S_2']]
df_d = df_d.drop(['customer_ID','S_2'], axis=1)
imputer.fit(df_d)
df_imp = pd.DataFrame(imputer.transform(df_d),columns = df_d.columns)
df_imp['index1'] = df_imp.index
df_customers['index1'] = df_customers.index
df_final = df_imp.merge(df_customers)
df_final = df_final.drop(['index','index1'], axis=1)
extracted_features_chunk = extract_features(df_final, column_id="customer_ID", column_sort="S_2",kind_to_fc_parameters=kind_to_fc_parameters_score_point1perc,n_jobs=12)
extracted_features_chunk.to_sql('train_tsfresh_imp',csv_database,if_exists = 'append')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 60/60 [24:01<00:00, 24.02s/it]
The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.


In [22]:
tsfresh_data = pd.read_sql_query('select * from train_tsfresh_imp',csv_database)

In [24]:
tsfresh_upd = tsfresh_data.set_index(['index'])

In [32]:
os.chdir(r'C:\Users\formy\Downloads\amex-default-prediction')
df_labels = pd.read_csv('train_labels.csv')

In [73]:
import lightgbm as lgb
rf1 = lgb.LGBMClassifier(boosting_type = 'DART',min_child_samples = 20,n_jobs = 12,importance_type = 'gain',n_estimators = 100,learning_rate = 0.225)
final_data_l = tsfresh_upd.merge(df_labels,left_index=True,right_on=['customer_ID'])
final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
import re
final_data_l  = final_data_l.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '1', x))
final_data_l = final_data_l.loc[:,~final_data_l.columns.duplicated()].copy()
X = final_data_l.iloc[:,final_data_l.columns != 'target']
X = X.drop(['customer_ID'],axis =1)
#X= X[important_feautures]
y = final_data_l[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y.squeeze(), test_size=0.33, random_state=21)
rf1.fit(X_train, y_train)
y_pred = rf1.predict_proba(X_test)[:,1]
y_true = y_test

In [74]:
print(' Metric for this column: ' +str(amex_metric(y_true, y_pred)))
sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
ColumnMap = {}
for col in extracted_features.columns:
    ColumnMap[re.sub('[^A-Za-z0-9_]+', '1', col)] = col
#Importance_data['feauture_v1'] = Importance_data[['feautures']].apply(lambda x:ColumnMap[x[0]] ,axis = 1)
#important_feautures = list(Importance_data.query('scores > 0.001')['feauture_v1'])

 Metric for this column: 0.7714295549455588


In [75]:
manual_data = pd.read_sql_query('select * from train_data_refined',csv_database)

In [88]:
trend =  manual_data[cols]

In [89]:
final_data_l

,D_56__permutation_entropy__dimension_3__tau_1,R_3__change_quantiles__f_agg_1mean1__isabs_True__qh_012__ql_010,D_79__quantile__q_017,S_12__autocorrelation__lag_8,D_129__max_langevin_fixed_point__m_3__r_30,D_129__quantile__q_019,D_54__energy_ratio_by_chunks__num_segments_10__segment_focus_1,D_83__autocorrelation__lag_9,B_5__spkt_welch_density__coeff_5,D_89__cwt_coefficients__coeff_6__w_5__widths_12151101201,...,D_48__mean,D_44__quantile__q_019,B_3__mean_change,R_1__mean,P_2__quantile__q_012,P_2__minimum,P_2__quantile__q_011,P_2__mean,customer_ID,target
0,1.720193,0.000000,0.004903,0.211674,1.006969,1.008457,0.154486,-0.801398,0.000262,0.014694,...,0.274552,0.007706,0.000205,0.004509,0.922758,0.868580,0.910803,0.933824,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,1.641735,0.001713,0.007028,0.030642,0.004905,0.009025,0.153201,-0.536296,0.006700,0.007072,...,0.048203,0.008515,-0.000064,0.006246,0.879521,0.861109,0.874581,0.899820,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,1.168518,0.000000,0.005263,-0.343246,0.010861,0.008118,0.153658,-0.177329,0.000021,0.013035,...,0.092284,0.008313,0.000198,0.006621,0.868611,0.797670,0.855280,0.878454,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,1.294545,0.003590,0.007423,-0.906943,1.009527,1.008506,0.153553,0.082370,0.003635,0.004146,...,0.172003,0.008605,-0.000082,0.005665,0.581358,0.567442,0.570931,0.598969,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,1.767761,0.000000,0.006361,-0.541641,1.006573,1.008552,0.154735,0.483887,0.000035,0.006430,...,0.253697,0.007276,-0.000033,0.004180,0.863347,0.805045,0.860932,0.891679,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,0.000000,0.001704,0.005404,-0.816608,0.005397,0.007245,0.154758,0.359917,0.020397,0.012592,...,0.033075,0.009080,-0.000166,0.005458,0.835549,0.730505,0.816762,0.848750,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,0.000000,0.000000,0.506794,0.671972,1.007672,1.006790,0.153870,0.014776,0.000013,0.013479,...,0.133014,0.132812,-0.005868,0.005542,0.850741,0.831279,0.840488,0.859327,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,1.767761,0.000199,0.005809,0.243341,1.007531,1.006191,0.154374,-0.612639,0.000627,0.012362,...,0.124234,0.008881,0.000275,0.004751,0.775580,0.756983,0.768935,0.786838,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,0.000000,0.001141,0.005740,-0.341802,0.006518,0.007269,0.153529,0.460363,0.000045,0.007214,...,0.326568,0.133043,-0.003780,0.023470,0.776380,0.754129,0.757363,0.804454,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [90]:
final_data_trend = final_data_l.merge(trend)

In [93]:
final_data_trend = final_data_trend.loc[:,~final_data_trend.columns.duplicated()].copy()
import re
final_data_trend  = final_data_trend.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '1', x))
final_data_trend = final_data_trend.loc[:,~final_data_trend.columns.duplicated()].copy()
X = final_data_trend.iloc[:,final_data_trend.columns != 'target']
X = X.drop(['customer_ID'],axis =1)
#X= X[important_feautures]
y = final_data_trend[['target']]
X_train, X_test, y_train, y_test = train_test_split(X, y.squeeze(), test_size=0.33, random_state=21)
rf1.fit(X_train, y_train)
y_pred = rf1.predict_proba(X_test)[:,1]
y_true = y_test

print(' Metric for this column: ' +str(amex_metric(y_true, y_pred)))
sorted_idx = rf1.feature_importances_.argsort()
# plt.barh(extracted_features.columns[sorted_idx], rf.feature_importances_[sorted_idx])
max_importance = max(rf1.feature_importances_[sorted_idx])
Importance_data = pd.DataFrame()
Importance_data['feautures'] = X_train.columns[sorted_idx]
Importance_data['scores'] = [i/max_importance for i in rf1.feature_importances_[sorted_idx]]
ColumnMap = {}
for col in extracted_features.columns:
    ColumnMap[re.sub('[^A-Za-z0-9_]+', '1', col)] = col

 Metric for this column: 0.7763212128662427


In [95]:
final_data_trend.columns

Index(['D_56__permutation_entropy__dimension_3__tau_1',
       'R_3__change_quantiles__f_agg_1mean1__isabs_True__qh_012__ql_010',
       'D_79__quantile__q_017', 'S_12__autocorrelation__lag_8',
       'D_129__max_langevin_fixed_point__m_3__r_30', 'D_129__quantile__q_019',
       'D_54__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
       'D_83__autocorrelation__lag_9', 'B_5__spkt_welch_density__coeff_5',
       'D_89__cwt_coefficients__coeff_6__w_5__widths_12151101201',
       ...
       'D_64_U_trend', 'D_66_010_trend', 'D_66_110_trend', 'D_68_010_trend',
       'D_68_110_trend', 'D_68_210_trend', 'D_68_310_trend', 'D_68_410_trend',
       'D_68_510_trend', 'D_68_610_trend'],
      dtype='object', length=496)